In [40]:
library(data.table)
library(stringr)
r.posts <- fread("posts.csv", na.strings = c("", " "))
r.replys <- fread("replys.csv", na.strings = c("", " "))
r.posts <- r.posts[postnums != 0]
nrow(r.posts)
luowen <- r.posts[guba_name == "螺纹钢吧"]
postnums.sum <- sum(r.posts$postnums)  # 统计所有postnums的总和
cat("螺纹钢吧帖子数量：", luowennum.count, "；postnums总和：", postnums.sum, "\n")
cat("两者对比：", ifelse(luowennum.count > postnums.sum, "螺纹钢吧帖子数量更大", "postnums总和更大"), "\n")
r.posts[, create_time := as.POSIXct(create_time)]
earl.posts <- r.posts[order(create_time), .SD[1], by = guba_name]
fwrite(earl.posts, "earl.posts.csv")
r.replys[, reply_time := as.POSIXct(reply_time)]
late.replys <- r.replys[order(post_id, reply_time), .SD[.N], by = post_id]
fwrite(late.replys, "late.replys.csv")
r.replys[, reply_content := str_remove_all(reply_content, "<[^>]+>")]
int.data <- merge(r.posts, r.replys, by = "post_id")
fwrite(int.data, "int.data.csv")
int.data[,create_date := as.Date(create_time,format = "%Y-%m-%d")]
int.data[, create_time := as.POSIXct(create_time,format="%Y-%m-%d %H:%M:%OS")]
min.date <- min(int.data$create_date)
max.date <- max(int.data$create_date)
guba.names <- unique(int.data$guba_name)
tim <- data.table(guba_name = rep(guba.names, each = as.integer(max.date - min.date + 1)),
create_date = rep(seq(min.date, max.date, by = "day"), length(guba.names)))
int.data.ts <- merge(tim, int.data, by = c("guba_name", "create_date"), all.x = TRUE)
int.data.ts[, `:=`(readnum = fifelse(is.na(readnum), 0, readnum), replynum = fifelse(is.na(replynum), 0, replynum))]
fwrite(int.data.ts, "int.data.ts.csv")
mx.post <- r.posts[order(-replynum)][1:10, .(post_id, guba_name, readnum, replynum)]
mx.ba <- r.posts[, .(total_readnum = sum(readnum), total_replynum = sum(replynum)), by = guba_name][order(-total_replynum)][1:10]
fwrite(mx.post, "mx.post.csv")
fwrite(mx.ba, "mx.ba.csv")
stst.int.post <- r.posts[, .(replynum_max = max(replynum),
replynum_min = min(replynum), 
replynum_mean = mean(replynum),
replynum_median = median(replynum), 
replynum_var = var(replynum),
readnum_max = max(readnum),
readnum_min = min(readnum), 
readnum_mean = mean(readnum), 
readnum_median = median(readnum), 
readnum_var = var(readnum))]
stst.int.ba <- r.posts[, 
.(replynum_max = max(replynum), 
replynum_min = min(replynum), 
replynum_mean = mean(replynum), 
replynum_median = median(replynum), 
replynum_var = var(replynum),
readnum_max = max(readnum), 
readnum_min = min(readnum), 
readnum_mean = mean(readnum), 
readnum_median = median(readnum), 
readnum_var = var(readnum)), by = guba_name]
fwrite(rbindlist(list(post = stst.int.post, ba = stst.int.ba, idcol = "level")), "stst.int.csv")



Warning message in fread("posts.csv", na.strings = c("", " ")):
"na.strings[2]==" " consists only of whitespace, ignoring. strip.white==TRUE (default) and "" is present in na.strings, so any number of spaces in string columns will already be read as <NA>."
Warning message in fread("posts.csv", na.strings = c("", " ")):
"Stopped early on line 42842. Expected 9 fields but found 1. Consider fill=TRUE and comment.char=. First discarded non-empty line: <<螺>>"
Warning message in fread("replys.csv", na.strings = c("", " ")):
"na.strings[2]==" " consists only of whitespace, ignoring. strip.white==TRUE (default) and "" is present in na.strings, so any number of spaces in string columns will already be read as <NA>."


[1] 42827

螺纹钢吧帖子数量： 19400 ；postnums总和： 695525208 
两者对比： postnums总和更大 


ERROR: Error in rbindlist(list(post = stst.int.post, ba = stst.int.ba, idcol = "level")): Item 2 has 11 columns, inconsistent with item 1 which has 10 columns. To fill missing columns use fill=TRUE.
